<a href="https://colab.research.google.com/github/stefanoridolfi/Coding_ML/blob/master/2_level__con_Bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:



commento="VERSIONE senza BIAS"
import matplotlib.pyplot as plt
import random
import numpy as np

from mpl_toolkits.mplot3d import Axes3D

from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


@np.vectorize
def generaPunti(n):
    l=[]
    for i in range(0, n+1):
        x = random.random()*300-150
        y = random.random()*300-150
        l.append([x,y])
    return l
def posizione(punto,xa,xb,ya,yb):
    if (punto[0]>xa and punto[0]<xb and punto[1]>ya and punto[1]<yb):
    #if (punto[0]>xa and  punto[1]<yb):
    #if (punto[0]>xa):
      formato = 'ro'
      out=1
    else:
      formato = 'go'
      out=0
    return out,formato

def posizione_neurale(p):
    if p>0.2:
      formato = 'ro'
      out=1
    else:
      formato = 'go'
      out=0
    return out,formato



def sigmoid(x):
    return 1 / (1 + np.e ** -x)
activation_function = sigmoid

'''
Creo matrice dei pesi. Input sono 3, x1,x2 e x3 che è BIAS. Il vettore X ha dimensioni 1x3. Lo strato hidden è composto da 4 uscite, y1,y2,y3 e y4 ed è rappresentato dal vettore Y di dimensioni 4x1
Il vettore Y, di dimensioni 4x1 è il risultato del prodotto Wih.dot.X La matrice Wih ha dimensioni 4x3. Y=Wih.dot.X (4x1=4x3.dot.3x1)
L'output Z ha dimensioni 1. La matrice dei pesi da Y a Z,matrice Who,  ha dimensione 1x4. L'output Z ha dimensioni 1x1. Per questo Z=Who.dot.Y (1x1=1x4.dot.4x1)  
'''
#Creo punti Input per training. Per ogni punto aggiungo un campo che vale 1 se il punto è nel quadrato, 0 se è fuori. Nel disegno 1 è punto rosso 0 è punto verde
# defnisco contorni del quadrato
xa=-50
xb=50
ya=-50
yb=50
learning_rate=0.1
n=20000
hid=5
training_set=[]
# Set di trainig formato da n punti
ListaPunti= generaPunti(n)
for punto in ListaPunti:
   training_set.append([punto[0],punto[1],posizione(punto,xa,xb,ya,yb)[0]])
# punto[2] è la posizione corretta del punto, che vale 1 se è all'interno del quadrato e 0 se è fuori il quadrato   
#print(training_set)

# Creo la matrice Wih, hid x2, i cui elementi sono i pesi. I pesi hanno un valore con distrubuzione gaussiano vm==0 e varianza=1
Wih=np.random.randn(hid,2)
# Creo la matrice Who, 1xhid, i cui elementi sono i pesi. I pesi hanno un valore distrubuzione gaussiano vm==0 e varianza=1
Who=np.random.randn(1,hid)
# per vedere le dimensioni di un array in comando è :array.shape, esempio Wih.shape
# Inizializzo i valori dell'input con una lista. X=(x1,x2).

# Creo il vettore BIAS del livello nascosto il vettore ha dimensioni (hidx1)
b1=[]
for t in range(0,hid):
  b1.append(1)
b1=np.array(b1)
b1=b1.reshape(hid,1)

#Creo il vettore BIAS del livello di uscita, il vettore ha dimensioni (1 x 1)
b2=[]
for t in range(0,1):
  b2.append(1)
b2=np.array(b2)
b2=b2.reshape(1,1)
###############TRAINING################################ 
print("INIZIO TRAINING SET\n")
epoche=100
Epo=[]
Err_y=[]

for e in range(1,epoche):
  Err=0
  Err_rel=0
  for punto in training_set:
    X=(punto[0],punto[1])
# lo devo trasformare in un array altrimenti non posso fare operazioni vettoriali
    X=np.array(X)
    X=X.reshape(2,1)
  

 
#calcolo Y=Wih.dot.X
    Y=np.dot(Wih,X)
#Sommo ad Y il BIAS b1
    Y=Y+b1
 
#calcolo Y_act che è il vettore Y a cui ho applicato la funzione di attivazione
    Y_act=activation_function(Y)
  
# Y ha dimensione 4x1
# Calcolo Z=Who.dot.Y.T
    Z=np.dot(Who,Y_act)
#Sommo ad Z il BIAS b2
    Z=Z+b2
#calcolo Z_act che è il vettore Z a cui ho applicato la funzione di attivazione
    Z_act=activation_function(Z)
 
  #print("Z_act= ", Z_act, "pos_reale=", punto[2])
#calcolo l'errore finale E=valore_corretto - Z_act . Il valore corretto è la terza colonna dei punti di training, ovvero training_set[2]
    error_output=punto[2]-Z_act
    Err=Err+(error_output)**2
    # print("Errore= ",output_error)
#Calcolo il delta da sommare al corrente valore di Y_act: moltiplicazione di learning_rate*output_errore*derivata_sigmoide in Z*valore del vettore Y_act 
# la derivata della sigmoide calcolata nel punto Z_act è: Z_act(1-Z_act)!!
    uno_Z_act=(1-Z_act)
    tmp1=error_output*Z_act*uno_Z_act
    # tmp1 + il delta del livello di uscita
    tmp2=learning_rate*(np.dot(tmp1,Y_act.T))
    # Calcolo d_b2
    d_b2=np.sum(tmp1,axis=1, keepdims=True)
  
  
  # Calcolo matrice Wih
    error_hidden=np.dot(Who.T,tmp1)
    tmp3=error_hidden*Y_act*(1-Y_act)
    #tmp3 è il delta del livello finale
    U=np.dot(tmp3,X.T)
     # Calcolo d_b1
    d_b1=np.sum(tmp3,axis=1, keepdims=True)
  #aggiorno matrice dei pesi Wih
    Wih=Wih+learning_rate*U
   #aggiorno matrice dei pesi Who
    Who=Who+tmp2
   #aggiorno d_b1
    b1=b1+d_b1*learning_rate
   #aggiorno b2
    b2=b2+d_b2*learning_rate
  #print("dim tmp=",tmp.shape,",dim Z_act=", Z_act.shape,",dim (1-Z_act)=",uno_Z_act.shape, ",dim Z_act_deri=",Z_act_deri.shape,",dim Delta_Who=",Delta_Who.shape)
  #print("Z_act=", Z_act,",(1-Z_act)=",uno_Z_act, ",Z_act_deri=",Z_act_deri,",Delta_Who=",Delta_Who)
  #print("Who=", Who)

  Err_rel=(Err)/len(training_set)
  Epo.append(e)
  Err_y.append(Err_rel)
  print("epoca=",e,"Errore =",Err_rel)

######################FINE TRAINING#######################################
# TEST SET  
print("INIZIO TEST\n")
t=1000
test_set=[]
# Set di trainig formato da n punti
test_set= generaPunti(t)

for punto in test_set:
  plt.plot(punto[0],punto[1],posizione(punto,xa,xb,ya,yb)[1])
plt.title("Disposizione corretta dei punti")
plt.show()


# Grafico punti dedotti da rete neurale
errori_tot=0
for punto in test_set:
   X=(punto[0],punto[1])
   X=np.array(X)
   X=X.reshape(2,1)
   Y=np.dot(Wih,X)+b1
   Y_act=activation_function(Y)
   Z=np.dot(Who,Y_act)+b2
   Z_act=activation_function(Z)
   plt.plot(punto[0],punto[1],posizione_neurale(Z_act)[1])
  # plt.title("Punti Dedotti dalla rete neurale")
#plt.show()

# Grafico errori
Px_in=[]
Px_out=[]
Py_in=[]
Py_out=[]
Zgraph_in=[]
Zgraph_out=[]

errori_tot=0
for punto in test_set:
   X=(punto[0],punto[1])
   if posizione(punto,xa,xb,ya,yb)[0]==1:
     Px_in.append(punto[0])
     Py_in.append(punto[1])
     
   else:
     Px_out.append(punto[0])
     Py_out.append(punto[1])
    

   X=np.array(X)
   X=X.reshape(2,1)
   Y=np.dot(Wih,X)+b1
   Y_act=activation_function(Y)
   Z=np.dot(Who,Y_act)+b2
   Z_act=activation_function(Z)
   if posizione(punto,xa,xb,ya,yb)[0]==1:
     Zgraph_in.append([Z_act])
   else:
     Zgraph_out.append([Z_act])
   #print("Z_act:posizione correta",posizione_neurale(Z_act)[0],":",posizione(punto,xa,xb,ya,yb)[0])
   #plt.plot(punto[0],punto[1],posizione_neurale(Z_act)[1])
   #print("Posizione, Z_act,Neurale, Errore Assoluto=",posizione(punto,xa,xb,ya,yb)[0],":",Z_act,":",posizione_neurale(Z_act)[0],":",posizione(punto,xa,xb,ya,yb)[0]-Z_act)
   if posizione_neurale(Z_act)[0]!=posizione(punto,xa,xb,ya,yb)[0]:
     plt.plot(punto[0],punto[1], 'bx')
     #pint("Z_act:formato neurale, output neurale, posizione corretta",Z_act,posizione_neurale(Z_act)[1],",",posizione_neurale(Z_act)[0],posizione(punto,xa,xb,ya,yb)[0])
     errori_tot=errori_tot+1
plt.title("Punti dedotti e punti errati in blu")
plt.show()

Px_in=np.array(Px_in)
Px_out=np.array(Px_out)
Py_in=np.array(Py_in)
Py_out=np.array(Py_out)
Zgraph_in=np.array(Zgraph_in)
Zgraph_out=np.array(Zgraph_out)
#Px_in=Px_in.reshape(t+1,1)
#Py_out=Py_out.reshape(t+1,1)
#Px_in=Px_out.reshape(t+1,1)
#Py_out=PY_out.reshape(t+1,1)
#Zgraph_in=Zgraph_in.reshape(t+1,1)
#Zgraph_out=Zgraph_out.reshape(t+1,1)
#print("inside=",len(Px_in))
#print("outside=",len(Px_out))



fig = plt.figure()
ax = fig.gca(projection='3d')
xdata = Px_in
ydata = Py_in
zdata = Zgraph_in
ax.scatter3D(xdata, ydata, zdata, s=30, c='r', marker='o');
xdata = Px_out
ydata = Py_out
zdata = Zgraph_out
ax.scatter3D(xdata, ydata, zdata, s=30, c='g', marker='^');
# Plot the surface.
plt.show()
print("Punti inside=",len(np.array(Px_in)))
print("Punti outside=",len(np.array(Px_out)))
print(commento," punti training=",n," epoche=",epoche," punti test=",t, " learning_rate=",learning_rate, " neuroni strato nascosto=",hid)
print("errori totali=",errori_tot, "su", t, "%",(errori_tot/t)*100)
Epo=np.array(Epo)
Epo=Epo.reshape(len(Epo))
Err_y=np.array(Err_y)
Err_y=Err_y.reshape(len(Err_y))
plt.plot(Epo, Err_y, 'r-')
plt.title('Errore relativi per epoche')
plt.show()